In [1]:
#ここではrulebase_datasetを用いて実際にテキストを生成していく

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import MeCab
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
import warnings
import math

warnings.filterwarnings("ignore", category=FutureWarning)
df_rules = pd.read_csv('/path/to/rulebase_dataset.csv')

In [3]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "retrieva-jp/t5-base-short"

# 転移学習済みモデル
MODEL_DIR = "/path/to/t5model"

In [4]:
#csvファイルの'input(ルールベース)'と'output'から各行を抽出したものをdf_rulesとする
def zenkaku_to_hankaku(text):
    zenkaku = 'ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'
    hankaku = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

    translation = str.maketrans(zenkaku, hankaku)
    return text.translate(translation)


all_data = []
for i in range(len(df_rules)):
    df_rule = df_rules.iloc[i,:]
    input = zenkaku_to_hankaku(df_rule['input'])
    output = zenkaku_to_hankaku(df_rule['output'])
    if len(input) > 0 and len(output) > 0:
        all_data.append({"input": input.lower(), "output": output.lower()})

In [5]:
#t5dataフォルダ内の空のtrain~test.tsvファイルにそれぞれ0.85:0.10:0.05の割合でデータを割り振る

import random
from tqdm import tqdm

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    input = data["input"]
    output = data["output"]
    assert len(input) > 0 and len(output) > 0
    return f"{input}\t{output}\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.85, 0.10, 0.05

with open(f"/home/tanaka/Archive/t5data/train.tsv", "w", encoding="utf-8") as f_train, \
    open(f"/home/tanaka/Archive/t5data/valid.tsv", "w", encoding="utf-8") as f_dev, \
    open(f"/home/tanaka/Archive/t5data/test.tsv", "w", encoding="utf-8") as f_test:

    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

1522it [00:00, 106878.24it/s]


In [6]:
#haggingfaceからT5トークナイザなどをインポート、シードを固定

import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

2024-03-19 10:25:05.635799: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 10:25:05.635850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 10:25:05.635870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 10:25:05.645249: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 10:25:06.744648: W tensorflow/compiler/

In [7]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/home/tanaka/Archive/t5data",  # データセットのディレクトリ
    model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    # max_input_length=512,
    # max_target_length=64,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    max_grad_norm=1.0,
    seed=42,
)

In [8]:
#ミニバッチ作成かつ、T5入力用にデータ整備（プ－リングなど）
    
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)

        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask,
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, inp, out):
        # ニュースタイトル生成タスク用の入出力形式に変換する。
        input = f"{inp}"
        target = f"{out}"
        return input, target

    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                inp = line[0]
                out = line[1]

                input, target = self._make_record(inp, out)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True,
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True,
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)

In [9]:
#ファインチューニングのためのクラス、pl.lightningを用いた
    
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None,
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked),
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters()
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters()
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparams.learning_rate,
                          eps=self.hparams.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer = self.optimizer, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=self.t_total)
        
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer,
            data_dir=args.data_dir,
            type_path=type_path,
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)

    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer,
                                                type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer,
                                            type_path="valid.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
                )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset,
                          batch_size=self.hparams.train_batch_size,
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset,
                          batch_size=self.hparams.eval_batch_size,
                          num_workers=4)



In [10]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  600,  # 入力文の最大トークン数
    "max_target_length": 80,  # 出力文の最大トークン数
    "train_batch_size":  8,  # 訓練時のバッチサイズ
    "eval_batch_size":   8,  # テスト時のバッチサイズ
    "num_train_epochs":  10,  # 訓練するエポック数
    })
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    devices=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    gradient_clip_val=args.max_grad_norm,
)

In [11]:
# 転移学習の実行（GPUを利用すれば1エポック2分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [12]:
#学習済みモデルを読み込む
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv",
                          input_max_len=args.max_input_length,
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

trained_model.eval()

inputs = []
outputs = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids,
        attention_mask=input_mask,
        max_length=args.max_target_length,
        temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
        repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True,
                            clean_up_tokenization_spaces=False)
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True,
                               clean_up_tokenization_spaces=False)
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True,
                               clean_up_tokenization_spaces=False)
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)


  0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
for output, target, input in zip(outputs, targets, inputs):
    print("generated: " + output.upper())
    print("actual:    " + target.upper())
    # print("body:      " + input)
    print()

generated: 湘南 キッカーは名古。右足を振り抜くが、シュートには持ち込めない
actual:    湘南 FKのキッカーの名古が右足を振り抜く。だが、相手の壁に防がれてしまう

generated: 鹿島 エヴェラウドがドリブルで持ち上がり、ペナルティエリア手前の中央から右足を振り抜く。しかし、シュートは枠の左にそれてしまう
actual:    鹿島 エヴェラウドがパスを受けると、フィジカルの強さを生かしてキープし、前を向く。そのまま持ち込んでペナルティエリア手前の中央からシュートを放つも、槙野にブロックされてしまう

generated: 鹿島 右CKを獲得する。キッカーの永戸は左足でアウトスイングのクロスを供給するが、ニアサイドで相手にクリアされてしまう
actual:    鹿島 右CKを獲得すると、キッカーは永戸。左足で低い弾道のクロスを入れるも、味方は触れない

generated: 神戸 古橋が自陣から右サイドへロングボールを送ると、藤本が抜け出す。藤本はペナルティエリア手前の中央まで駆け上がると、そのままドリブルで持ち上がり、左サイドの敵陣深くからオフサイドの判定を受ける
actual:    神戸 後方でのパス回しから、味方が右サイドの敵陣中央へ縦パスを送る。藤本が抜け出して収めるが、オフサイドの判定となってしまう

generated: 名古屋 吉田豊が左サイドの敵陣中央からペナルティエリア手前の中央へスルーパスを出す。東が反応するが、ボールはゴールラインを割ってしまう
actual:    名古屋 吉田豊が左サイドの敵陣中央からペナルティエリア左のディフェンスラインの裏へ縦パスを入れる。しかし、走り出した柿谷へは合わない

generated: 大分 福森が左サイドの敵陣深くでボールを持ち、DFをかわしてシュートを放つ。しかし、Jシミッチに当たってしまい、ゴールには至らない
actual:    大分 敵陣でボールをつなぐと、最後は町田からのパスを受けた福森が仕掛け、ペナルティエリア手前の左からシュートを放つ。しかし、相手に当たり、枠をとらえられない

generated: 名古屋 味方がペナルティエリア手前の右から左足で浮き球を送ると、山崎がダイレクトでシュートを放つ。しかし、枠をとらえられない
actual:    名古屋 山崎が